# Install Dependencies and Mount Drive

In [1]:
import pandas as pd
import json
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set Paths

In [3]:
# Shared Drive
shared_drive_path = '/content/drive/Shareddrives/NMA-DL/'

# Images (JPG)
imagesAdd_test_path = shared_drive_path + 'images/additional_images/test_images/'
imagesAdd_train_path = shared_drive_path + 'images/additional_images/trainval_images/'
imagesVCOCO_test_path = shared_drive_path + 'images/vcoco/test_images/'
imagesVCOCO_train_path = shared_drive_path + 'images/vcoco/trainval_images/'

# Annotations (JSON)
annotations_path = shared_drive_path + 'annotations/'
annotations_path_test = annotations_path + 'initial_annotations/test/'
annotations_path_train = annotations_path + 'initial_annotations/trainval/'
annotations_path_test_vcoco = annotations_path + 'initial_annotations/test_vcoco/'
annotations_path_train_vcoco = annotations_path + 'initial_annotations/trainval_vcoco/'

# Subject-Verb-Target Annotations (CSV)
#subject_verb_target_test_path = annotations_path + 'subject_verb_target_test.csv'
#object_id_test_path = annotations_path + 'object_id_test.csv'
#subject_verb_target_train_path = annotations_path + 'subject_verb_target_train.csv'
#object_id_train_path = annotations_path + 'object_id_train.csv'

# Image IDs
imageID_test_path = annotations_path + 'images_test.ids'
imageID_train_path = annotations_path + 'images_trainval.ids'

# Other MetaData (file names mapped to H2OI IDs) and Categories
metaData_path = annotations_path + 'metadata.json'
categories_path = annotations_path + 'categories.json'

# Load Image IDs

In [4]:
# Load Test IDs
imageID_test_ids = open(imageID_test_path,'r')
imageID_test = imageID_test_ids.read().split('\n')[:-1]
imageID_test = pd.DataFrame().assign(imageID=imageID_test,Dataset='test')
imageID_test_ids.close()

# Load Train IDs
imageID_train_ids = open(imageID_train_path,'r')
imageID_train = imageID_train_ids.read().split('\n')[:-1]
imageID_train = pd.DataFrame().assign(imageID=imageID_train,Dataset='train')
imageID_train_ids.close()

# Combine DataFrames
image_IDs = pd.concat([imageID_test,imageID_train],ignore_index=True).rename(columns={'imageID': 'Image_ID'})
image_IDs = image_IDs.assign(Id=image_IDs['Image_ID'].str.replace('HH','').str.replace('HO','').astype(int))
image_IDs = image_IDs.assign(VCOCO=image_IDs['Image_ID'].str.find('HO') == 0)
image_IDs

,Image_ID,Dataset,Id,VCOCO
0,HH0000000004,test,4,False
1,HH0000000005,test,5,False
2,HH0000000008,test,8,False
3,HH0000000018,test,18,False
4,HH0000000049,test,49,False
...,...,...,...,...
15758,HO0000579891,train,579891,True
15759,HO0000580381,train,580381,True
15760,HO0000580396,train,580396,True
15761,HO0000581446,train,581446,True


# Load File Info from Directories

## Images (.JPG)

In [5]:
# Get file names from nested directories
files_imagesAdd_test_df = pd.DataFrame().assign(fileName=os.listdir(imagesAdd_test_path),Dataset='test',VCOCO=False,pathName=imagesAdd_test_path)
files_imagesAdd_train_df = pd.DataFrame().assign(fileName=os.listdir(imagesAdd_train_path),Dataset='train',VCOCO=False,pathName=imagesAdd_train_path)
files_imagesVCOCO_test_df = pd.DataFrame().assign(fileName=os.listdir(imagesVCOCO_test_path),Dataset='test',VCOCO=True,pathName=imagesVCOCO_test_path)
files_imagesVCOCO_train_df = pd.DataFrame().assign(fileName=os.listdir(imagesVCOCO_train_path),Dataset='train',VCOCO=True,pathName=imagesVCOCO_test_path)

# Combine file names
files_images = pd.concat([files_imagesAdd_test_df,files_imagesAdd_train_df,files_imagesVCOCO_test_df,files_imagesVCOCO_train_df],ignore_index=True)
files_images = files_images.assign(Image_ID=files_images['fileName'].str.replace('.jpg',''))
files_images = files_images[files_images['fileName'].str.find('H') == 0]
files_images = files_images[['Image_ID','Dataset','fileName','pathName','VCOCO']]

files_images

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


,Image_ID,Dataset,fileName,pathName,VCOCO
0,HH0000023656,test,HH0000023656.jpg,/content/drive/Shareddrives/NMA-DL/images/addi...,False
1,HH0000012116,test,HH0000012116.jpg,/content/drive/Shareddrives/NMA-DL/images/addi...,False
2,HH0000011555,test,HH0000011555.jpg,/content/drive/Shareddrives/NMA-DL/images/addi...,False
3,HH0000040383,test,HH0000040383.jpg,/content/drive/Shareddrives/NMA-DL/images/addi...,False
4,HH0000037130,test,HH0000037130.jpg,/content/drive/Shareddrives/NMA-DL/images/addi...,False
...,...,...,...,...,...
13967,HO0000455334,train,HO0000455334.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,True
13968,HO0000368775,train,HO0000368775.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,True
13969,HO0000144582,train,HO0000144582.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,True
13970,HO0000142440,train,HO0000142440.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,True


## Annotations (.JSON)

In [6]:
# Load File Paths in Initial_Annotations Directory
files_train = pd.DataFrame().assign(fileName=os.listdir(annotations_path_train),pathName=annotations_path_train,Dataset='train')
files_test = pd.DataFrame().assign(fileName=os.listdir(annotations_path_test),pathName=annotations_path_test,Dataset='test')
files_annotations = pd.concat([files_train,files_test],ignore_index=True)
files_annotations = files_annotations.assign(Image_ID=files_annotations['fileName'].str.replace('.json',''))
files_annotations = files_annotations.assign(fullPath=files_annotations['pathName'].str.cat(files_annotations['fileName']))
files_annotations = files_annotations[['Image_ID','Dataset','fileName','pathName','fullPath']]

files_annotations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,Image_ID,Dataset,fileName,pathName,fullPath
0,HO0000145351,train,HO0000145351.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
1,HO0000035093,train,HO0000035093.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
2,HH0000023264,train,HH0000023264.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
3,HH0000021938,train,HH0000021938.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
4,HH0000030312,train,HH0000030312.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
...,...,...,...,...,...
13951,HO0000007938,test,HO0000007938.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
13952,HO0000521689,test,HO0000521689.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
13953,HO0000097865,test,HO0000097865.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...
13954,HO0000021905,test,HO0000021905.json,/content/drive/Shareddrives/NMA-DL/annotations...,/content/drive/Shareddrives/NMA-DL/annotations...


# Load Raw Annotations

In [7]:
# Load Annotations from JSON Files to DataFrame
if os.path.exists(annotations_path + 'initial_annotations_raw.csv'):
  annotations_raw = pd.read_csv(annotations_path + 'initial_annotations_raw.csv')
else:
  annotations_raw = pd.DataFrame()
  for file in files_annotations['fullPath']:
    annotations_raw = annotations_raw.append(pd.json_normalize(json.load(open(file))['entities']),ignore_index=True)
  annotations_raw.to_csv(annotations_path + 'initial_annotations_raw.csv',index=False)

annotations_raw

,id,sourceId,category,actions,geometry.geometryType,geometry.isNormalized,geometry.vertices,name,history
0,378574,HO0000145351,chair,[],1,True,"[0.8404575163398693, 0.42022875816993466, 0.99...",NaN,NaN
1,426906,HO0000145351,person,"[{'value': 'still', 'targetId': '378574', 'ins...",1,True,"[0, 0.0022549019607843133, 0.87640522875817, 0...",NaN,NaN
2,537880,HO0000145351,person,"[{'value': 'still', 'targetId': '537880', 'ins...",1,True,"[0.21333333333333335, 0, 0.3506372549019608, 0...",NaN,NaN
3,542353,HO0000145351,person,"[{'value': 'still', 'targetId': '542353', 'ins...",1,True,"[0.1619934640522876, 0.08034313725490197, 0.23...",NaN,NaN
4,543183,HO0000145351,person,"[{'value': 'still', 'targetId': '543183', 'ins...",1,True,"[0.1061764705882353, 0.07640522875816994, 0.17...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
128859,484579,HO0000021905,person,"[{'value': 'jump/fall', 'targetId': '616366', ...",1,True,"[0.32278125, 0.1283817427385892, 0.42700000000...",NaN,NaN
128860,616366,HO0000021905,snowboard,[],1,True,"[0.289984375, 0.3401244813278008, 0.4370468750...",NaN,NaN
128861,297952,HO0000268541,tie,[],1,True,"[0.3663023255813953, 0.7595581395348837, 0.485...",NaN,NaN
128862,542744,HO0000268541,person,"[{'value': 'undet_motion', 'targetId': '542744...",1,True,"[0.0022093023255813954, 0.22858139534883723, 0...",NaN,NaN


# Process Annotations

## Extract Object Info (ID, Category, Location, Source Image)

In [8]:
# Extract Object ID Annotations
annotations_objects = annotations_raw.rename(columns={'sourceId': 'Image_ID', 'id': 'Object_ID','category': 'Object_Category', 'geometry.vertices': 'Bounding_Box'})
annotations_objects = annotations_objects[['Image_ID','Object_ID','Object_Category','Bounding_Box']]

annotations_objects

,Image_ID,Object_ID,Object_Category,Bounding_Box
0,HO0000145351,378574,chair,"[0.8404575163398693, 0.42022875816993466, 0.99..."
1,HO0000145351,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0..."
2,HO0000145351,537880,person,"[0.21333333333333335, 0, 0.3506372549019608, 0..."
3,HO0000145351,542353,person,"[0.1619934640522876, 0.08034313725490197, 0.23..."
4,HO0000145351,543183,person,"[0.1061764705882353, 0.07640522875816994, 0.17..."
...,...,...,...,...
128859,HO0000021905,484579,person,"[0.32278125, 0.1283817427385892, 0.42700000000..."
128860,HO0000021905,616366,snowboard,"[0.289984375, 0.3401244813278008, 0.4370468750..."
128861,HO0000268541,297952,tie,"[0.3663023255813953, 0.7595581395348837, 0.485..."
128862,HO0000268541,542744,person,"[0.0022093023255813954, 0.22858139534883723, 0..."


In [9]:
# Track Duplicates
annotations_objects[annotations_objects.duplicated(subset=['Object_ID'],keep='first')]

,Image_ID,Object_ID,Object_Category,Bounding_Box
6927,HH0000011001,g66xke1cfbuafln6y03s,person,"[0.671875, 0.40833333134651184, 0.895312488079..."
6928,HH0000011001,kdc9mb5vu3blbnmkkzdh,person,"[0.29218751192092896, 0.4208333194255829, 0.69..."
8151,HO0000048595,208666,person,"[0.168015625, 0.2998958333333333, 0.2294062499..."
8152,HO0000048595,214934,person,"[0.3335625, 0.45837500000000003, 0.42357812499..."
8153,HO0000048595,220389,person,"[0.4108125, 0.442625, 0.5051875, 0.55020833333..."
...,...,...,...,...
128815,HO0000006864,543001,person,"[0.6538906250000001, 0.2966276346604215, 0.678..."
128816,HO0000006864,544329,person,"[0.789484375, 0.29505854800936765, 0.849718749..."
128817,HO0000006864,1182940,suitcase,"[0.3969375, 0.5688056206088993, 0.487703125, 0..."
128818,HO0000006864,1735124,person,"[0.94284375, 0.38997658079625297, 1, 0.7802810..."


In [10]:
# Drop duplicates
annotations_objects = annotations_objects.drop_duplicates(subset=['Object_ID'])
annotations_objects

,Image_ID,Object_ID,Object_Category,Bounding_Box
0,HO0000145351,378574,chair,"[0.8404575163398693, 0.42022875816993466, 0.99..."
1,HO0000145351,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0..."
2,HO0000145351,537880,person,"[0.21333333333333335, 0, 0.3506372549019608, 0..."
3,HO0000145351,542353,person,"[0.1619934640522876, 0.08034313725490197, 0.23..."
4,HO0000145351,543183,person,"[0.1061764705882353, 0.07640522875816994, 0.17..."
...,...,...,...,...
128859,HO0000021905,484579,person,"[0.32278125, 0.1283817427385892, 0.42700000000..."
128860,HO0000021905,616366,snowboard,"[0.289984375, 0.3401244813278008, 0.4370468750..."
128861,HO0000268541,297952,tie,"[0.3663023255813953, 0.7595581395348837, 0.485..."
128862,HO0000268541,542744,person,"[0.0022093023255813954, 0.22858139534883723, 0..."


## Extract Subject-Action-Target Info

In [11]:
def hasAction(action):
  hasAction = False
  if isinstance(action, str) | isinstance(action,list):
    if len(action) > 0:
      hasAction = True
  return hasAction

In [12]:
# Extract Action Annotations
if os.path.exists(annotations_path + 'annotations_actions.csv'):
  annotations_actions = pd.read_csv(annotations_path + 'annotations_actions.csv')
else:
  annotations_actions = pd.DataFrame()
  for action in annotations_raw['actions'][annotations_raw['actions'].apply(hasAction)]:
    # If annotations_raw.csv was used, string needs to be converted to dictionary
    if os.path.exists(annotations_path + 'initial_annotations_raw.csv'):
      action = json.loads(action.replace('\'','\"'))
    annotations_actions = annotations_actions.append(pd.json_normalize(action))
  annotations_actions = annotations_actions.rename(columns={'value': 'Action', 'targetId': 'Target_ID', 'instrumentId': 'Subject_ID'})
  annotations_actions.to_csv(annotations_path + 'annotations_actions.csv',index=False)

annotations_actions

,Action,Target_ID,Subject_ID
0,still,378574,426906
1,eat,1539613,705982
2,hold,705982,426906
3,sit,378574,426906
4,still,537880,537880
...,...,...,...
148811,other,616366,484579
148812,undet_motion,542744,542744
148813,hold,668788,542744
148814,drink,668788,668788


In [13]:
# Track Duplicates
annotations_actions[annotations_actions.duplicated(keep='first')]

,Action,Target_ID,Subject_ID
3702,board,2202030,479325
8043,use_on,1088679,2104079
8160,undet_motion,g66xke1cfbuafln6y03s,g66xke1cfbuafln6y03s
8161,stand,g66xke1cfbuafln6y03s,g66xke1cfbuafln6y03s
8162,undet_motion,kdc9mb5vu3blbnmkkzdh,kdc9mb5vu3blbnmkkzdh
...,...,...,...
148769,stand,544329,544329
148770,undet_motion,1735124,1735124
148771,stand,1735124,1735124
148772,walk,2166522,2166522


In [14]:
# Drop duplicates
annotations_actions = annotations_actions.drop_duplicates()
annotations_actions

,Action,Target_ID,Subject_ID
0,still,378574,426906
1,eat,1539613,705982
2,hold,705982,426906
3,sit,378574,426906
4,still,537880,537880
...,...,...,...
148811,other,616366,484579
148812,undet_motion,542744,542744
148813,hold,668788,542744
148814,drink,668788,668788


# Combine DataFrames

In [15]:
# Combine Action with Subject Info
annotations = annotations_actions.merge(annotations_objects,how='left',left_on='Subject_ID',right_on='Object_ID').drop(columns='Object_ID')
annotations = annotations.rename(columns={'Object_Category': 'Subject_Category', 'Bounding_Box': 'Subject_Bounding_Box'})

# Combine Action with Target Info
annotations = annotations.merge(annotations_objects.drop(columns='Image_ID'),how='left',left_on='Target_ID',right_on='Object_ID')
annotations = annotations.rename(columns={'Object_Category': 'Target_Category', 'Bounding_Box': 'Target_Bounding_Box'}).drop(columns='Object_ID')

# Add Image Info
annotations = annotations.merge(image_IDs,on='Image_ID',copy=False)
annotations

,Action,Target_ID,Subject_ID,Image_ID,Subject_Category,Subject_Bounding_Box,Target_Category,Target_Bounding_Box,Dataset,Id,VCOCO
0,still,378574,426906,HO0000145351,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",chair,"[0.8404575163398693, 0.42022875816993466, 0.99...",train,145351,True
1,eat,1539613,705982,HO0000145351,spoon,"[0.2654575163398693, 0.3678921568627451, 0.482...",bowl,"[0.3070424836601307, 0.755686274509804, 0.7247...",train,145351,True
2,hold,705982,426906,HO0000145351,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",spoon,"[0.2654575163398693, 0.3678921568627451, 0.482...",train,145351,True
3,sit,378574,426906,HO0000145351,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",chair,"[0.8404575163398693, 0.42022875816993466, 0.99...",train,145351,True
4,still,537880,537880,HO0000145351,person,"[0.21333333333333335, 0, 0.3506372549019608, 0...",person,"[0.21333333333333335, 0, 0.3506372549019608, 0...",train,145351,True
...,...,...,...,...,...,...,...,...,...,...,...
142192,other,616366,484579,HO0000021905,person,"[0.32278125, 0.1283817427385892, 0.42700000000...",snowboard,"[0.289984375, 0.3401244813278008, 0.4370468750...",test,21905,True
142193,undet_motion,542744,542744,HO0000268541,person,"[0.0022093023255813954, 0.22858139534883723, 0...",person,"[0.0022093023255813954, 0.22858139534883723, 0...",test,268541,True
142194,hold,668788,542744,HO0000268541,person,"[0.0022093023255813954, 0.22858139534883723, 0...",cup,"[0.3424883720930233, 0.3524651162790698, 0.529...",test,268541,True
142195,drink,668788,668788,HO0000268541,cup,"[0.3424883720930233, 0.3524651162790698, 0.529...",cup,"[0.3424883720930233, 0.3524651162790698, 0.529...",test,268541,True


In [16]:
def getVerbType(action):
  '''Define the verb type as posture, motion, object interaction, social interaction, or violent interaction'''

  if action in ['bend','stand','sit','other','lay','undet_pos','crouch']:
    verbType = 'posture'
  elif action in ['still','swim','ride','climb','walk','dance','board','run','crawl','jump/fall','undet_motion']:
    verbType = 'motion'
  elif action in ['hold','lift','smoke','carry_no_hands','point_sth','use_on','drink','talk on phone','eat','watch','manipulate','pull/push_soft','aim/shoot']:
    verbType = 'object_interaction'
  elif action in ['hug','wave','kiss','act_on','thumbsup','highfive','pat','hold_sb','carry','pull_sb_soft','handshake','fistbump','point_sb']:
    verbType = 'social_interaction'
  elif action in ['punch','kick','choke','block','catch','pull/push_strong','throw','hit_with']:
    verbType = 'violent_interaction'
  else:
    verbType = 'error'
    
  return verbType

In [17]:
# Add verb type to DataFrame and Reorder
annotations = annotations.assign(VerbType=annotations['Action'].apply(getVerbType))
annotations = annotations[['Id','Image_ID','Dataset','VCOCO','VerbType','Action','Subject_ID','Subject_Category','Subject_Bounding_Box','Target_ID','Target_Category','Target_Bounding_Box']]

annotations

,Id,Image_ID,Dataset,VCOCO,VerbType,Action,Subject_ID,Subject_Category,Subject_Bounding_Box,Target_ID,Target_Category,Target_Bounding_Box
0,145351,HO0000145351,train,True,motion,still,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",378574,chair,"[0.8404575163398693, 0.42022875816993466, 0.99..."
1,145351,HO0000145351,train,True,object_interaction,eat,705982,spoon,"[0.2654575163398693, 0.3678921568627451, 0.482...",1539613,bowl,"[0.3070424836601307, 0.755686274509804, 0.7247..."
2,145351,HO0000145351,train,True,object_interaction,hold,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",705982,spoon,"[0.2654575163398693, 0.3678921568627451, 0.482..."
3,145351,HO0000145351,train,True,posture,sit,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",378574,chair,"[0.8404575163398693, 0.42022875816993466, 0.99..."
4,145351,HO0000145351,train,True,motion,still,537880,person,"[0.21333333333333335, 0, 0.3506372549019608, 0...",537880,person,"[0.21333333333333335, 0, 0.3506372549019608, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
142192,21905,HO0000021905,test,True,posture,other,484579,person,"[0.32278125, 0.1283817427385892, 0.42700000000...",616366,snowboard,"[0.289984375, 0.3401244813278008, 0.4370468750..."
142193,268541,HO0000268541,test,True,motion,undet_motion,542744,person,"[0.0022093023255813954, 0.22858139534883723, 0...",542744,person,"[0.0022093023255813954, 0.22858139534883723, 0..."
142194,268541,HO0000268541,test,True,object_interaction,hold,542744,person,"[0.0022093023255813954, 0.22858139534883723, 0...",668788,cup,"[0.3424883720930233, 0.3524651162790698, 0.529..."
142195,268541,HO0000268541,test,True,object_interaction,drink,668788,cup,"[0.3424883720930233, 0.3524651162790698, 0.529...",668788,cup,"[0.3424883720930233, 0.3524651162790698, 0.529..."


In [35]:
# Add image file data
annotations = annotations.merge(files_images,how='left',on=['Image_ID','VCOCO','Dataset'])

annotations

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'fileName_x', 'pathName_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


,Id,Image_ID,Dataset,VCOCO,VerbType,Action,Subject_ID,Subject_Category,Subject_Bounding_Box,Target_ID,...,fileName_y,pathName_y,fileName_x,pathName_x,fileName_y,pathName_y,fileName_x,pathName_x,fileName_y,pathName_y
0,145351,HO0000145351,train,True,motion,still,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",378574,...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
1,145351,HO0000145351,train,True,object_interaction,eat,705982,spoon,"[0.2654575163398693, 0.3678921568627451, 0.482...",1539613,...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
2,145351,HO0000145351,train,True,object_interaction,hold,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",705982,...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
3,145351,HO0000145351,train,True,posture,sit,426906,person,"[0, 0.0022549019607843133, 0.87640522875817, 0...",378574,...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
4,145351,HO0000145351,train,True,motion,still,537880,person,"[0.21333333333333335, 0, 0.3506372549019608, 0...",537880,...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000145351.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142192,21905,HO0000021905,test,True,posture,other,484579,person,"[0.32278125, 0.1283817427385892, 0.42700000000...",616366,...,HO0000021905.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000021905.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000021905.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000021905.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000021905.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
142193,268541,HO0000268541,test,True,motion,undet_motion,542744,person,"[0.0022093023255813954, 0.22858139534883723, 0...",542744,...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...
142194,268541,HO0000268541,test,True,object_interaction,hold,542744,person,"[0.0022093023255813954, 0.22858139534883723, 0...",668788,...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-DL/images/vcoc...,HO0000268541.jpg,/content/drive/Shareddrives/NMA-

In [36]:
annotations_count = annotations[annotations['VerbType']=='social_interaction'].groupby('Image_ID').count()
annotations_count[annotations_count['VerbType']==1]

,Id,Dataset,VCOCO,VerbType,Action,Subject_ID,Subject_Category,Subject_Bounding_Box,Target_ID,Target_Category,...,fileName_y,pathName_y,fileName_x,pathName_x,fileName_y,pathName_y,fileName_x,pathName_x,fileName_y,pathName_y
Image_ID,,,,,,,,,,,,,,,,,,,,,
HH0000000160,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HH0000000196,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HH0000000206,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HH0000000214,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HH0000000482,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HO0000570471,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HO0000570474,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HO0000573214,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [37]:
annotations.groupby(['Image_ID','VerbType']).count()

Id  Dataset  VCOCO  Action  Subject_ID  \
Image_ID     VerbType                                                     
HH0000000005 motion               4        4      4       4           4   
             posture              4        4      4       4           4   
             social_interaction   4        4      4       4           4   
HH0000000006 motion              12       12     12      12          12   
             object_interaction   2        2      2       2           2   
...                              ..      ...    ...     ...         ...   
HO0000581657 object_interaction   9        9      9       9           9   
             posture              9        9      9       9           9   
HO0000581860 motion               1        1      1       1           1   
             object_interaction   2        2      2       2           2   
             posture              1        1      1       1           1   

                                 Subject_Category  Subject_Bounding_Box  \
Image_ID     VerbType                                                     
HH0000000005 motion                             4                     4   
             posture                            4                     4   
             social_interaction                 4                     4   
HH0000000006 motion                            12                    12   
             object_interaction                 2                     2   
...                                           ...                   ...   
HO0000581657 object_interaction                 9                     9   
             posture                            9                     9   
HO0000581860 motion                             1                     1   
             object_interaction                 2                     2   
             posture                            1                     1   

                                 Target_ID  Target_Category  \
Image_ID     VerbType                                         
HH0000000005 motion                      4                4   
             posture                     4                4   
             social_interaction          4                4   
HH0000000006 motion                     12               12   
             object_interaction          2                2   
...                                    ...              ...   
HO0000581657 object_interaction          9                9   
             posture                     9                9   
HO0000581860 motion                      1                1   
             object_interaction          2                2   
             posture                     1                1   

                                 Target_Bounding_Box  ...  fileName_y  \
Image_ID     VerbType                                 ...               
HH0000000005 motion                                4  ...           4   
             posture                               4  ...           4   
             social_interaction                    4  ...           4   
HH0000000006 motion                               12  ...          12   
             object_interaction                    2  ...           2   
...                                              ...  ...         ...   
HO0000581657 object_interaction                    9  ...           9   
             posture                               9  ...           9   
HO0000581860 motion                                1  ...           1   
             object_interaction                    2  ...           2   
             posture                               1  ...           1   

                                 pathName_y  fileName_x  pathName_x  \
Image_ID     VerbType                                                 
HH0000000005 motion                       4           4           4   
             posture                      4           4           4   
             social_interac

In [19]:
images_notInData = list(set(annotations['Image_ID']) - set(files_images['Image_ID']))
images_notInAnnotations = list(set(files_images['Image_ID']) - set(annotations['Image_ID']))
print('Image IDs with Annotations but no Images = ',len(images_notInData))
print('Example IDs:',images_notInData[:4])
print('\nImage IDs with Images but no Annotations = ',len(images_notInAnnotations))
print('Example IDs:',images_notInAnnotations[:4])

Image IDs with Annotations but no Images =  15
Example IDs: ['HH0000009302', 'HH0000034471', 'HH0000005026', 'HH0000024648']

Image IDs with Images but no Annotations =  45
Example IDs: ['HO0000378614', 'HO0000226928', 'HO0000011696', 'HO0000413805']


In [20]:
def checkAnnotation(Image_ID):
  if Image_ID in images_notInAnnotations:
    return False
  else:
    return True

In [21]:
temp = files_images.assign(HasAnnotations=files_images['Image_ID'].apply(checkAnnotation))
temp[temp['HasAnnotations']].groupby(['Dataset','VCOCO']).count()

Image_ID  fileName  pathName  HasAnnotations
Dataset VCOCO                                              
test    False       726       726       726             726
        True       4919      4919      4919            4919
train   False      2898      2898      2898            2898
        True       5382      5382      5382            5382

In [22]:
# Save DataFrame to csv
annotations.to_csv(annotations_path + 'annotations.csv',index=False)

# Load Other MetaData Files from H2O Dataset

In [23]:
# Load object categories
categories_json = json.load(open(categories_path))
categories = np.array(categories_json)

print(categories)

['person' 'bicycle' 'car' 'motorcycle' 'airplane' 'bus' 'train' 'truck'
 'boat' 'traffic light' 'fire hydrant' 'street sign' 'stop sign'
 'parking meter' 'bench' 'bird' 'cat' 'dog' 'horse' 'sheep' 'cow'
 'elephant' 'bear' 'zebra' 'giraffe' 'hat' 'backpack' 'umbrella' 'shoe'
 'eye glasses' 'handbag' 'tie' 'suitcase' 'frisbee' 'skis' 'snowboard'
 'sports ball' 'kite' 'baseball bat' 'baseball glove' 'skateboard'
 'surfboard' 'tennis racket' 'bottle' 'plate' 'wine glass' 'cup' 'fork'
 'knife' 'spoon' 'bowl' 'banana' 'apple' 'sandwich' 'orange' 'broccoli'
 'carrot' 'hot dog' 'pizza' 'donut' 'cake' 'chair' 'couch' 'potted plant'
 'bed' 'mirror' 'dining table' 'window' 'desk' 'toilet' 'door' 'tv'
 'laptop' 'mouse' 'remote' 'keyboard' 'cell phone' 'microwave' 'oven'
 'toaster' 'sink' 'refrigerator' 'blender' 'book' 'clock' 'vase'
 'scissors' 'teddy bear' 'hair drier' 'toothbrush' 'hair brush' 'banner'
 'blanket' 'branch' 'bridge' 'building-other' 'bush' 'cabinet' 'cage'
 'cardboard' 'carpet' '

In [ ]:
# Load Train and Test "Instances"
instances_test_vcoco_json = json.load(open(annotations_path_test_vcoco + '/instances_test.json'))
instances_test_vcoco_df = pd.DataFrame(instances_test_vcoco_json['annotations']).assign(dataset='test')
instances_train_vcoco_json = json.load(open(annotations_path_train_vcoco + '/instances_trainval.json'))
instances_train_vcoco_df = pd.DataFrame(instances_train_vcoco_json['annotations']).assign(dataset='train')

instances_vcoco_df = pd.concat([instances_test_vcoco_df,instances_train_vcoco_df],ignore_index=True)
instances_vcoco_df

,area,bbox,category_id,id,image_id,iscrowd,dataset
0,1.114440e+06,"[958, 1323, 753, 1480]",1,969966,HH0000000005,0,test
1,1.229193e+06,"[325, 1207, 763, 1611]",1,770585,HH0000000005,0,test
2,2.390850e+06,"[2393, 1227, 1518, 1575]",1,213718,HH0000000005,0,test
3,1.679236e+06,"[1430, 1308, 1118, 1502]",1,740431,HH0000000005,0,test
4,1.402310e+05,"[105, 6, 299, 469]",1,802101,HH0000000008,0,test
...,...,...,...,...,...,...,...
128714,1.089993e+03,"[128, 120, 40, 27]",62,232274,HO0000581657,0,train
128715,1.570450e+03,"[1, 123, 44, 36]",62,16715,HO0000581657,0,train
128716,3.188088e+03,"[421, 236, 59, 54]",1,594087,HO0000581657,0,train
128717,3.231629e+04,"[76, 150, 211, 153]",54,724245,HO0000581860,0,train


In [ ]:
# Load Train and Test "Interactions"
interactions_test_vcoco_json = json.load(open(annotations_path_test_vcoco + '/interactions_test.json'))
interactions_train_vcoco_json = json.load(open(annotations_path_train_vcoco + '/interactions_trainval.json'))

for keys in interactions_test_vcoco_json[0]:
  print(keys)

for keys in interactions_train_vcoco_json[0]:
  print(keys)

action_name
ann_id
image_id
label
role_name
role_object_id
action_name
ann_id
image_id
label
role_name
role_object_id


In [ ]:
# Load Meta Data for Non-V-COCO images
metaData_json = json.load(open(metaData_path))
metaData = pd.DataFrame(metaData_json['images']).set_index('Id')

metaData

,File name,Height,Interaction,Source,Url,Width,MS-COCO Id,Original file name,Frame Id
Id,,,,,,,,,
17675,017675.jpg,624,Dancing,Website: www.flickr.com,https://c1.staticflickr.com/3/2921/14632395758...,1024,NaN,NaN,NaN
30002,030002.jpg,424,Kissing,Existing Datasets : MS-COCO,http://farm8.staticflickr.com/7213/6981041686_...,640,524436.0,NaN,NaN
9292,009292.jpg,381,Handshake,Existing Datasets : MS-COCO,http://farm9.staticflickr.com/8197/8229677645_...,640,81885.0,NaN,NaN
17851,017851.jpg,210,Highfive,Existing Datasets : Human-Interaction-Images,https://vision.cs.hacettepe.edu.tr/interaction...,300,NaN,HighFive/175.jpg,NaN
13554,013554.jpg,640,Holding,Existing Datasets : MS-COCO,http://farm4.staticflickr.com/3457/3886408866_...,480,214528.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
18321,018321.jpg,580,Giving,Microsoft's Cognitive Services APIs - Bing search,https://pattydoc.com/wp-content/uploads/2019/0...,640,NaN,NaN,NaN
18943,018943.jpg,720,Giving,Microsoft's Cognitive Services APIs - Bing search,https://www.wowamazing.com/wp-content/uploads/...,1280,NaN,NaN,NaN
25639,025639.jpg,801,Giving,Microsoft's Cognitive Services APIs - Bing search,https://www.wowamazing.com/wp-content/uploads/...,1200,NaN,NaN,NaN
